In [ ]:
import numpy as np
import numpy.linalg as la
import pandas as pd

# Parameter

In [ ]:
# Jumlah grid
NX = 100
NY = 100

# panjang interval 
LX = 10
LY = 10

# Panjang grid
dx = LX/NX
dy = LY/NY

# titik tengah
x0 = LX/2
y0 = LY/2

# data input
X = np.linspace(0, LX, NX+1) 
Y = np.linspace(0, LY, NY+1)

x, y = np.meshgrid(X, Y)

# potensial OH
V = 0.5*(x - x0)**2 + 0.5*(y - y0)**2

a = 2/np.sqrt(LX*LY)
psx, psy = np.pi/LX, np.pi/LY

# Fungsi Gelombag

In [ ]:
def psi(m, n):
    """
        LX = batas sumbu x
        LY = batas sumbu y
        m = indeks sumbu x
        n = indeks sumbu y
    """
    # fungsi gelombang
    psi = a * np.sin(m*psx*x) * np.sin(n*psy*y)
    
    return psi

In [ ]:
def d2psi(m, n):
    
    psix = -0.5 * -(m*psx)**2 * a * np.sin(m*psx*x) * np.sin(n*psy*y)
        
    psiy = -0.5 * -(n*psy)**2 * a * np.sin(m*psx*x) * np.sin(n*psy*y)
    
    return psix, psiy

# Index

In [ ]:
ind = np.zeros([100, 2], dtype="int32")
i = 0
for p in range(1, 10+1):
    for q in range(1, 10+1):
        ind[i, 0] = p
        ind[i, 1] = q
        i += 1

# Energi Kinetik

In [ ]:
ekinetik = np.zeros([NX, NY])

for ii in range(NX):
    m = ind[ii, 0]
    n = ind[ii, 1]
    
    psi_kiri = psi(m, n)
    
    for jj in range(NY):
        p = ind[jj, 0]
        q = ind[jj, 1]
        
        element = 0.
        psi_kanan = psi(p, q)
        d2psix, d2psiy = d2psi(p, q)

        for i in range(1, NX):
            for j in range(1, NY):

                element += psi_kiri[i,j]*(d2psix[i,j] + d2psiy[i,j])

        element *= dx*dy
        ekinetik[ii, jj] = element

# Energi Kinetik + Potensial

In [ ]:
# tempat menyimpan nilai
coef_fourier = np.array([])
coef_ekspansi = np.array([])
ground_state = np.array([])

k = 0
jumlah_data = 4

while(k < jumlah_data):
    
    # jumlah koefisien fourier baris x kolom
    baris = 5
    kolom = 5

    cnm = np.zeros([baris, kolom])

    # nilai koefisien fourier
    # bilangan random 
    for i in range(baris):
        for j in range(kolom):
            cnm[i, j] = np.random.uniform(-1, 1)

    # potensial random
    V = 0.
    for m in range(baris):
        for n in range(kolom):
            V += cnm[m,n] * np.sin((m+1)*np.pi*x/LX) * np.sin((n+1)*np.pi*y/LY)

    # supaya nilai potensial positif
    V = V - V.min()

    # tempat menyimpan nilai
    rep_matriks = np.zeros([NX, NY])
    
    for ii in range(NX):
        m = ind[ii, 0]
        n = ind[ii, 1]

        psi_kiri = psi(m, n)

        for jj in range(NY):
            p = ind[jj, 0]
            q = ind[jj, 1]

            element = 0.
            psi_kanan = psi(p, q)

            for i in range(1, NX):
                for j in range(1, NY):

                    element += psi_kiri[i,j]*V[i,j]*psi_kanan[i,j]

            element *= dx*dy
            rep_matriks[ii, jj] += ekinetik[ii, jj] + element
            
    # menghitung nilai eigen
    w, v = la.eig(rep_matriks)
    
    # ground state energi
    gs = np.min(w)
    igs = np.argmin(w)
    
    # koefisien ekspansi ground state
    cn = v[:, igs]
    
    # flatten koefisien fourier
    cnm = cnm.flatten()
    
    # simpan nilai 
    coef_fourier = np.append(coef_fourier, cnm)
    coef_ekspansi = np.append(coef_ekspansi, cn)
    ground_state = np.append(ground_state, gs)
    
    k += 1

In [ ]:
# 1 baris 1 data

coef_fourier2 = coef_fourier.reshape(jumlah_data, baris*kolom)
coef_ekspansi2 = coef_ekspansi.reshape(jumlah_data, NX)
ground_state2 = ground_state.reshape(jumlah_data, 1)

# Simpan Nilai

In [ ]:
# simpan nilai
Li = []
for i in range(25):
    a = "c" + str(i+1)
    Li.append(a)

# ekspor data koefisien
df_1 = pd.DataFrame(data = coef_fourier2, columns = Li)
df_1.to_csv("/content/drive/MyDrive/Data_Colab/coef_fourier.csv", index = False, header = True)

In [ ]:
# simpan nilai
Li = []
for i in range(100):
    a = "c" + str(i+1)
    Li.append(a)

# ekspor data koefisien
df_2 = pd.DataFrame(data = coef_ekspansi2, columns = Li)
df_2.to_csv("/content/drive/MyDrive/Data_Colab/coef_ekspansi.csv", index = False, header = True)

In [ ]:
# ekspor data koefisien
df_2 = pd.DataFrame(data = ground_state, columns = ['Energi'])
df_2.to_csv("/content/drive/MyDrive/Data_Colab/ground_state.csv", index = False, header = True)